<h2>
# Movies Classification Project
</center></h4> <h4>
 <img src="http://usblogs.pwc.com/emerging-technology/wp-content/uploads/2017/07/deep-learning-900x280.png" >


# Machine Learning Text

</h4>

Developed by [Sergio Castelblanco](http://www.linkedin.com/in/sergio-castelblanco/) - [Jesus Solano](http://www.jesussolano.com)

Andes University

version 2.1, July 9 2018

In [0]:
!pip install tqdm
!pip install livelossplot

In [0]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm
import os
%matplotlib inline

# Read Data

In [0]:
# Creates dataframes for images indices.

dataTraining = pd.read_csv("dataTraining.csv",index_col=0)
dataTesting = pd.read_csv("dataTesting.csv",index_col=0)

In [9]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [10]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# Read Text

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

maxWords= 5000

vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=maxWords)

tokenText = vectorizer.fit_transform(dataTraining['plot'])
testingText = vectorizer.fit_transform(dataTesting['plot'])
tokenText.shape

(7895, 5000)

In [0]:
# Get labels for each image. 

dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
yGenres = le.fit_transform(dataTraining['genres'])

In [13]:
# Creates datasets for training and testing.   
xTrainTxt, xValidationTxt, yTrainTxt, yValidationTxt = train_test_split(tokenText, yGenres, test_size=0.20, random_state=22)
xTrainTxt.shape

(6316, 5000)

# Machine Learning Models

### Random Forest

In [21]:
# Creates a random forest classifier. 

mcRandomTxt = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=22,verbose=0))

# Train the model. 

mcRandomTxt.fit(xTrainTxt,  yTrainTxt)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=22, verbose=0, warm_start=False),
          n_jobs=1)

In [22]:
# Score the AUC over the test set. 

y_pred_genres = mcRandomTxt.predict_proba(xValidationTxt)

mcRandomTxt_AUC = roc_auc_score(yValidationTxt, y_pred_genres, average='macro')

print('Random Forest model score is: ' ,mcRandomTxt_AUC, 'AUC.')

Random Forest model score is:  0.8155028010105557 AUC.


### Gradient Boosting

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

# Creates a GradientBoostingClassifier. 

mcGradientTxt = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, max_depth=10, random_state=22,verbose=0))

# Train the model. 

mcGradientTxt.fit(xTrainTxt,  yTrainTxt)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=22, subsample=1.0, verbose=0,
              warm_start=False),
          n_jobs=1)

In [27]:
# Score the AUC over the test set. 

y_pred_genres = mcGradientTxt.predict_proba(xValidationTxt)

mcGradientTxt_AUC = roc_auc_score(yValidationTxt, y_pred_genres, average='macro')

print('Gradient Boosting score is: ' ,mcGradientTxt_AUC, 'AUC.')

Gradient Boosting score is:  0.7651230922952764 AUC.


# Apply models to test

In [0]:
# Print prediction for test (kaggle) data. 

yPredTest = mcRandomTxt.predict_proba(testingText)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(yPredTest, index=dataTesting.index, columns=cols).to_csv('pred_genres_RF_img.csv', index_label='ID')

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: 